In [70]:
import json
import time

In [71]:
class Base:
  def success(self, message: str, data: list = None):
    response = {"status": "success", "message": message}
    if data is not None:
      response["data"] = data
    return json.dumps(response, ensure_ascii=False)

  def error(self, message: str):
    return json.dumps({"status": "error", "message": message}, ensure_ascii=False)

In [72]:
class ShoppingCart(Base):
  def __init__(self):
    self.shopping_carts = []

  def add_cart(self, member_id: int):
    cart_id = len(self.shopping_carts) + 1
    member_id = member_id
    current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    data = {
      'cart_id': cart_id,
      'member_id': member_id,
      'create_time': current_time
    }
    self.shopping_carts.append(data)

  def get_carts(self):
    print(self.success("", self.shopping_carts))

# 測試
if __name__ == "__main__":
  shopping_cart_instance = ShoppingCart()
  shopping_cart_instance.add_cart(1)
  shopping_cart_instance.add_cart(2)
  shopping_cart_instance.add_cart(3)
  shopping_cart_instance.get_carts()

{"status": "success", "message": "", "data": [{"cart_id": 1, "member_id": 1, "create_time": "2024-12-12 07:45:28"}, {"cart_id": 2, "member_id": 2, "create_time": "2024-12-12 07:45:28"}, {"cart_id": 3, "member_id": 3, "create_time": "2024-12-12 07:45:28"}]}


In [73]:
class Product(Base):
  def __init__(self):
    self.products = []

  def add_product(self, product_serialize: str):
    try:
      product = json.loads(product_serialize)
      product_id = len(self.products) + 1
      current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
      product['product_id'] = product_id
      product['create_time'] = current_time
      product['last_update_time'] = current_time
      self.products.append(product)
      print(self.success("產品新增成功", product))
    except json.JSONDecodeError:
      print(self.error("產品資料格式不正確"))

  def get_product(self, product_id: int, output: bool = True):
    for product in self.products:
      if product['product_id'] == product_id:
        if output: print(self.success("", product))
        return product
    if output: print(self.error(f"沒找到產品編號 {product_id}"))
    return {}

# 測試
if __name__ == "__main__":
  product_instance = Product()

  # 新增產品測試
  product_instance.add_product('{"product_name": "電視機", "product_price": 50000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/電視機.png"}')
  product_instance.add_product('{"product_name": "冷氣", "product_price": 120000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/冷氣.png"}')

  # 取得產品測試
  product_instance.get_product(1)
  product_instance.get_product(3)


{"status": "success", "message": "產品新增成功", "data": {"product_name": "電視機", "product_price": 50000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/電視機.png", "product_id": 1, "create_time": "2024-12-12 07:45:32", "last_update_time": "2024-12-12 07:45:32"}}
{"status": "success", "message": "產品新增成功", "data": {"product_name": "冷氣", "product_price": 120000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/冷氣.png", "product_id": 2, "create_time": "2024-12-12 07:45:32", "last_update_time": "2024-12-12 07:45:32"}}
{"status": "success", "message": "", "data": {"product_name": "電視機", "product_price": 50000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/電視機.png", "product_id": 1, "create_time": "2024-12-12 07:45:32", "last_update_time": "2024-12-12 07:45:32"}}
{"status": "error", "message": "

In [75]:
class ShoppingCartItem(Base):
  def __init__(self, product_instance: str):
    self.cart_items = {}
    self.cart_item_id = 0
    self.product_instance = product_instance

  def add_cart_item(self, cart_id: int, product_id: int, quantity: int = 1):
    # 初始化該購物車的商品列表
    if cart_id not in self.cart_items:
        self.cart_items[cart_id] = []

    # 檢查購物車內是否已有相同產品
    for item in self.cart_items[cart_id]:
      if item['product_id'] == product_id:
        print(self.error("產品已存在於購物車"))
        return

    # 從 Product 實例取得產品資料
    product = self.product_instance.get_product(product_id, False)

    # 如果產品尚未加入購物車
    self.cart_item_id += 1
    current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    new_item = {
      'cart_id': cart_id,
      'product_id': product_id,
      'cart_item_id': self.cart_item_id,
      'cart_item_serialize': product,
      'cart_item_price': product["product_price"],
      'cart_item_qty': quantity,
      'cart_item_screenshot_url': product["product_screenshot_url"],
      'cart_item_status': product["product_status"],
      'create_time': current_time,
      'last_update_time': current_time
    }
    self.cart_items[cart_id].append(new_item)
    print(self.success("產品已加入購物車", new_item))

  def remove_cart_item(self, cart_id: int, product_id: int):
    if cart_id in self.cart_items:
      filtered_items = [
        item for item in self.cart_items[cart_id] if item['product_id'] != product_id
      ]
      if len(filtered_items) < len(self.cart_items[cart_id]):
        self.cart_items[cart_id] = filtered_items
        print(self.success(f"產品編號 {product_id} 已從購物車編號 {cart_id} 移除"))
      else:
        print(self.error(f"購物車編號 {cart_id} 中找不到產品編號 {product_id}"))
    else:
      print(self.error(f"找不到購物車編號 {cart_id}"))

  def get_cart_item(self, cart_item_id: int):
    for cart_id, items in self.cart_items.items():
      for item in items:
        if item['cart_item_id'] == cart_item_id:
          print(self.success("找到購物車項目", item))
          return
    print(self.error(f"找不到購物車項目 {cart_item_id}"))

  def sync_cart_item_status(self, cart_id: int):
    if cart_id not in self.cart_items:
      print(self.error(f"找不到購物車編號 {cart_id}"))
      return

    updated_items = []
    for item in self.cart_items[cart_id]:
      # 取得最新的產品資料
      product = None
      for prod in self.product_instance.products:
        if prod["product_id"] == item["product_id"]:
          product = prod
          break

      item["cart_item_status"] = product.get("product_status")
      item["last_update_time"] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
      updated_items.append(item)

    # 更新購物車的項目列表
    self.cart_items[cart_id] = updated_items
    print(self.success(f"購物車編號 {cart_id} 的所有商品狀態已同步", updated_items))

  def update_cart_item_qty(self, cart_item_id: int, cart_item_qty: int):
    # 確保數量有效
    if cart_item_qty <= 0:
      print(self.error("數量必須大於零"))
      return

    # 遍歷購物車查找目標項目
    for cart_id, items in self.cart_items.items():
      for item in items:
        if item['cart_item_id'] == cart_item_id:
          # 更新數量與最後更新時間
          item['cart_item_qty'] = cart_item_qty
          item['last_update_time'] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
          print(self.success(f"購物車項目 {cart_item_id} 數量已更新為 {cart_item_qty}", item))
          return

    # 若找不到項目
    print(self.error(f"找不到購物車項目 {cart_item_id}"))

  def get_cart_items(self, cart_id: int):
    print(self.success("取得所有購物車項目", self.cart_items))

  def empty_cart_items(self, cart_id: int):
    if cart_id in self.cart_items:
      self.cart_items[cart_id] = []
      print(self.success(f"購物車編號 {cart_id} 的所有項目已清空"))
    else:
      print(self.error(f"找不到購物車編號 {cart_id}"))

# 測試
if __name__ == "__main__":
  shopping_cart_item_instance = ShoppingCartItem(product_instance)
  shopping_cart_item_instance.add_cart_item(shopping_cart_instance.shopping_carts[0]["cart_id"], product_instance.products[0]["product_id"], 1)
  shopping_cart_item_instance.add_cart_item(shopping_cart_instance.shopping_carts[0]["cart_id"], product_instance.products[0]["product_id"], 2)
  shopping_cart_item_instance.add_cart_item(shopping_cart_instance.shopping_carts[0]["cart_id"], product_instance.products[0]["product_id"]+1, 1)
  time.sleep(1)
  shopping_cart_item_instance.update_cart_item_qty(shopping_cart_item_instance.cart_items[shopping_cart_instance.shopping_carts[0]["cart_id"]][0]["cart_item_id"], 3)
  shopping_cart_item_instance.get_cart_item(1)
  shopping_cart_item_instance.sync_cart_item_status(1)
  shopping_cart_item_instance.get_cart_items(shopping_cart_instance.shopping_carts[0]["cart_id"])
  shopping_cart_item_instance.empty_cart_items(shopping_cart_instance.shopping_carts[0]["cart_id"])
  shopping_cart_item_instance.get_cart_items(shopping_cart_instance.shopping_carts[0]["cart_id"])


{"status": "success", "message": "產品已加入購物車", "data": {"cart_id": 1, "product_id": 1, "cart_item_id": 1, "cart_item_serialize": {"product_name": "電視機", "product_price": 50000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/電視機.png", "product_id": 1, "create_time": "2024-12-12 07:45:32", "last_update_time": "2024-12-12 07:45:32"}, "cart_item_price": 50000, "cart_item_qty": 1, "cart_item_screenshot_url": "/project/電視機.png", "cart_item_status": "active", "create_time": "2024-12-12 07:46:34", "last_update_time": "2024-12-12 07:46:34"}}
{"status": "error", "message": "產品已存在於購物車"}
{"status": "success", "message": "產品已加入購物車", "data": {"cart_id": 1, "product_id": 2, "cart_item_id": 2, "cart_item_serialize": {"product_name": "冷氣", "product_price": 120000, "product_init_qty": 10, "product_remaining_qty": 10, "product_status": "active", "product_screenshot_url": "/project/冷氣.png", "product_id": 2, "create_time": "2024-12-12 07:4

In [76]:
class Member(Base):
  members = []  # 用於存儲所有會員的資料
  histories = {}  # 用於存儲會員的歷史紀錄
  next_id = 1  # 用於生成唯一的會員ID

  def __init__(self):
    self.member_user_id = None
    self.member_password = None
    self.member_email = None
    self.member_name = None

  def is_user_registered(self, member_user_id: str):
    return any(member['member_user_id'] == member_user_id for member in self.members)

  def register(self, member_user_id: str, password: str, member_name: str, member_email: str):
    if not all([member_user_id, password, member_name, member_email]):
      return self.error("註冊失敗，會員資料不完整")

    if self.is_user_registered(member_user_id):
      return self.error("註冊失敗，會員已註冊")

    member_data = {
      'member_user_id': member_user_id,
      'member_password': password,
      'member_email': member_email,
      'member_name': member_name
    }
    self.members.append(member_data)
    return self.success(f"{member_user_id} 成功加入會員")

  def login(self, member_user_id: str, password: str):
    for member in self.members:
      if member['member_user_id'] == member_user_id and member['member_password'] == password:
        return self.success("驗證成功")
    return self.error("驗證失敗")

  def add_member(self, member_data: dict):
    if not isinstance(member_data, dict):
      return self.error("無效的會員資料格式")

    member_id = self.next_id
    self.next_id += 1

    member_data = {
      'member_id': member_id,
      'member_user_id': member_data.get("member_user_id", ""),
      'member_password': member_data.get("member_password", ""),
      'member_email': member_data.get("member_email", ""),
      'member_name': member_data.get("member_name", "")
    }
    self.members.append(member_data)
    return self.success("會員新增成功", member_data)

  def update_member(self, member_user_id: str, updated_data: dict):
    for member in self.members:
      if member['member_user_id'] == member_user_id:
        member.update(updated_data)
        return self.success("會員更新成功", member)
    return self.error("會員未找到")

  def reset_password(self, member_user_id: str, password: str):
    for member in self.members:
      if member['member_user_id'] == member_user_id:
        member['member_password'] = password
        return self.success("密碼重設成功")
    return self.error("會員未找到，重設失敗")

  def add_history(self, member_user_id: str, product_id: str):
    if not isinstance(product_id, str):
      return self.error("產品ID格式無效")

    for member in self.members:
      if member['member_user_id'] == member_user_id:
        if member_user_id not in self.histories:
          self.histories[member_user_id] = []
          self.histories[member_user_id].append({
            "product_id": product_id,
            "time": time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
          })
          return self.success("歷史紀錄新增成功")
    return self.error("會員未找到，新增失敗")

# 測試
if __name__ == "__main__":
  # 測試註冊
  member_instance = Member()
  print(member_instance.register("user001", "securepassword", "John Doe", "johndoe@example.com"))

  # 測試登入
  print(member_instance.login("user001", "securepassword"))

  # 測試新增會員
  new_member_data = '{"member_password": "123456", "member_user_id": "user002", "member_email": "user002@example.com", "member_name": "Jane Doe"}'
  print(member_instance.add_member(new_member_data))

  # 測試更新會員資料
  print(member_instance.update_member(1, '{"member_name": "John Smith"}'))

  # 測試重設密碼
  print(member_instance.reset_password("user001", "newpassword123"))

  # 測試新增歷史紀錄
  print(member_instance.add_history("user001", 101))

  # 測試註冊成功
  registration_success1 = member_instance.register(
    member_user_id="user123",
    password="securepassword",
    member_name="John Doe",
    member_email="johndoe@example.com"
  )
  print(registration_success1)  # Output: user123 成功加入會員

  # 測試註冊失敗（會員已存在）
  registration_failure_duplicate = member_instance.register(
    member_user_id="user123",
    password="anotherpassword",
    member_name="Jane Doe",
    member_email="janedoe@example.com"
  )
  print(registration_failure_duplicate)  # Output: 註冊失敗，會員已註冊

  # 測試登入成功
  login_success = member_instance.login(member_user_id="user123", password="securepassword")
  print(login_success)  # Output: 驗證成功

  # 測試登入失敗（密碼錯誤）
  login_failure_wrong_password = member_instance.login(member_user_id="user123", password="wrongpassword")
  print(login_failure_wrong_password)  # Output: 驗證失敗

  # 測試登入失敗（會員 ID 錯誤）
  login_failure_wrong_user_id = member_instance.login(member_user_id="user456", password="securepassword")
  print(login_failure_wrong_user_id)  # Output: 驗證失敗

  # 測試註冊失敗（缺少必要參數）
  registration_failure_missing_info = member_instance.register(
    member_user_id="user456",
    password="anotherpassword",
    member_name="",  # 缺少名稱
    member_email="user456@example.com"
  )
  print(registration_failure_missing_info)  # Output: 註冊失敗，會員資料不完整


{"status": "success", "message": "user001 成功加入會員"}
{"status": "success", "message": "驗證成功"}
{"status": "error", "message": "無效的會員資料格式"}
{"status": "error", "message": "會員未找到"}
{"status": "success", "message": "密碼重設成功"}
{"status": "error", "message": "產品ID格式無效"}
{"status": "success", "message": "user123 成功加入會員"}
{"status": "error", "message": "註冊失敗，會員已註冊"}
{"status": "success", "message": "驗證成功"}
{"status": "error", "message": "驗證失敗"}
{"status": "error", "message": "驗證失敗"}
{"status": "error", "message": "註冊失敗，會員資料不完整"}


In [77]:
class HistoryRecord(Base):
  def __init__(self, member_id: int, product_id: int):
    self.member_id = member_id
    self.product_id = product_id
    self.current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

  def to_dict(self):
    return {
      "member_id": self.member_id,
      "product_id": self.product_id,
      "create_time": self.current_time
    }

In [78]:
class HistoryManager(Base):
  def __init__(self):
    self.history_records = []  # 存儲所有歷史記錄

  def add_history(self, member_id: int, product_id: int):
    if not isinstance(member_id, int) or not isinstance(product_id, int):
      return self.error("member_id 和 product_id 必須是整數")

    record = HistoryRecord(member_id, product_id)
    self.history_records.append(record)
    return self.success("歷史記錄新增成功", record.to_dict())

  def get_member_history(self, member_id: int):
    if not isinstance(member_id, int):
      return self.error("member_id 必須是整數")

    member_history = [record.to_dict() for record in self.history_records if record.member_id == member_id]
    if member_history:
      return self.success("成功獲取會員歷史記錄", member_history)
    return self.error(f"未找到 member_id 為 {member_id} 的歷史記錄")

  def get_all_histories(self):
    all_histories = [record.to_dict() for record in self.history_records]
    return self.success("所有歷史記錄", all_histories)

if __name__ == "__main__":
  # 測試
  history_instance = HistoryManager()

  # 新增歷史記錄
  print(history_instance.add_history(1, 101))
  print(history_instance.add_history(2, 102))
  print(history_instance.add_history(1, 103))

  # 獲取特定會員的歷史記錄
  print(history_instance.get_member_history(1))
  print(history_instance.get_member_history(2))

  # 獲取所有歷史記錄
  print(history_instance.get_all_histories())


{"status": "success", "message": "歷史記錄新增成功", "data": {"member_id": 1, "product_id": 101, "create_time": "2024-12-12 07:47:08"}}
{"status": "success", "message": "歷史記錄新增成功", "data": {"member_id": 2, "product_id": 102, "create_time": "2024-12-12 07:47:08"}}
{"status": "success", "message": "歷史記錄新增成功", "data": {"member_id": 1, "product_id": 103, "create_time": "2024-12-12 07:47:08"}}
{"status": "success", "message": "成功獲取會員歷史記錄", "data": [{"member_id": 1, "product_id": 101, "create_time": "2024-12-12 07:47:08"}, {"member_id": 1, "product_id": 103, "create_time": "2024-12-12 07:47:08"}]}
{"status": "success", "message": "成功獲取會員歷史記錄", "data": [{"member_id": 2, "product_id": 102, "create_time": "2024-12-12 07:47:08"}]}
{"status": "success", "message": "所有歷史記錄", "data": [{"member_id": 1, "product_id": 101, "create_time": "2024-12-12 07:47:08"}, {"member_id": 2, "product_id": 102, "create_time": "2024-12-12 07:47:08"}, {"member_id": 1, "product_id": 103, "create_time": "2024-12-12 07:47:08"}]}

In [79]:
class Order(Base):
  def __init__(self, order_id):
    self.order_id = order_id
    self.products = []
    self.status = "待處理"  # 訂單狀態可以是“待處理”、“已送貨”、“已退貨”

  def add_product(self, product):
    if product:
      self.products.append(product)
      return self.success("產品新增至訂單成功", product)
    return self.error("無效的產品，無法加入訂單")

  def ship_order(self):
    if self.status == "待處理":
      self.status = "已送貨"
      return self.success(f"訂單 {self.order_id} 已送貨")
    return self.error(f"訂單狀態無法送貨，當前狀態為：{self.status}")

  def return_order(self):
    if self.status == "已送貨":
      self.status = "已退貨"
      return self.success(f"訂單 {self.order_id} 已退貨")
    return self.error(f"訂單狀態無法退貨，當前狀態為：{self.status}")

  def get_order_info(self):
    if not self.products:
      return self.error("訂單內無任何產品")
    product_info = ', '.join(
      [f"{p['product_name']} (ID: {p['product_id']}, Price: ${p['product_price']})" for p in self.products]
    )
    return self.success(f"Order ID: {self.order_id}, Status: {self.status}, Products: [{product_info}]")


if __name__ == "__main__":
  # Create an order and add products
  order_instance = Order("ORD001")
  order_instance.add_product(product_instance.get_product(1, False))
  order_instance.add_product(product_instance.get_product(2, False))

  # Test the order methods
  print(order_instance.get_order_info())  # 顯示訂單資訊。
  print(order_instance.ship_order())    # 應顯示訂單Order001已出貨。"
  print(order_instance.get_order_info())  # 更新訂單資訊。
  print(order_instance.return_order())   # 應顯示Order ORD001已經被退貨。"
  print(order_instance.get_order_info())  # 更新訂單資訊。


{"status": "success", "message": "Order ID: ORD001, Status: 待處理, Products: [電視機 (ID: 1, Price: $50000), 冷氣 (ID: 2, Price: $120000)]"}
{"status": "success", "message": "訂單 ORD001 已送貨"}
{"status": "success", "message": "Order ID: ORD001, Status: 已送貨, Products: [電視機 (ID: 1, Price: $50000), 冷氣 (ID: 2, Price: $120000)]"}
{"status": "success", "message": "訂單 ORD001 已退貨"}
{"status": "success", "message": "Order ID: ORD001, Status: 已退貨, Products: [電視機 (ID: 1, Price: $50000), 冷氣 (ID: 2, Price: $120000)]"}


In [82]:
# 通用參數介面
class IPaymentParameters:
  pass

# 貨到付款參數
class CashOnDeliveryParameters(IPaymentParameters):
  pass

# 信用卡支付參數
class CreditCardPaymentParameters(IPaymentParameters):
  def __init__(self, card_number, expiry_year, expiry_month, security_code):
    self.card_number = card_number
    self.expiry_year = expiry_year
    self.expiry_month = expiry_month
    self.security_code = security_code

# 匯款支付參數
class RemittancePaymentParameters(IPaymentParameters):
  def __init__(self, last_five_digits, remitter, remitter_contact_info):
    self.last_five_digits = last_five_digits
    self.remitter = remitter
    self.remitter_contact_info = remitter_contact_info

# 行動支付參數
class MobilePaymentParameters(IPaymentParameters):
  def __init__(self, mobile_pay):
    self.mobilepay = mobile_pay

In [83]:
# 行動支付接口
class IMobilePay:
  def __init__(self):
    self.mobile_payment_name = ""

  def pay(self):
    raise NotImplementedError

# 行動支付方式：LinePay
class LinePay(IMobilePay):
  def __init__(self):
    super().__init__()
    self.mobile_payment_name = "LinePay"

  def pay(self):
    return True

# 行動支付方式：JkoPay
class JkoPay(IMobilePay):
  def __init__(self):
    super().__init__()
    self.mobile_payment_name = "JkoPay"

  def pay(self):
    return True

In [84]:
# 支付方法的抽象基類
class PaymentMethod:
  def __init__(self):
    self.payment_name = ""

  # 通用支付方法，接受參數物件
  def pay(self, parameters: IPaymentParameters):
    raise NotImplementedError

# 貨到付款實作
class CashOnDelivery(PaymentMethod):
  def __init__(self):
    super().__init__()
    self.payment_name = "CashOnDelivery"

  def pay(self, parameters: IPaymentParameters):
    print("\n收到款項")
    return True

# 信用卡支付實作
class CreditCardPayment(PaymentMethod):
  def __init__(self):
    super().__init__()
    self.payment_name = "CreditCardPayment"
    self.last_four_digits = ""

  def pay(self, parameters: IPaymentParameters):
    credit_card_params = parameters
    # 保存信用卡後四碼
    self.last_four_digits = credit_card_params.card_number[-4:]
    print(f"\n付款成功信用卡後四碼：{self.last_four_digits}")
    return True

# 匯款支付實作
class RemittancePayment(PaymentMethod):
  def __init__(self):
    super().__init__()
    self.payment_name = "RemittancePayment"
    self.receiving_bank = "800"  # 假設預設銀行代碼
    self.receiving_account_number = "1234567890"  # 假設預設帳號

  def pay(self, parameters: IPaymentParameters):
    remittance_params = parameters

    print("\n回報匯款資訊")
    print(f"匯款帳號後五碼：{remittance_params.last_five_digits}")
    print(f"匯款人姓名：{remittance_params.remitter}")
    print(f"匯款人聯絡方式：{remittance_params.remitter_contact_info}")

    # 保存匯款的資料
    self.last_five_digits = remittance_params.last_five_digits
    self.remitter = remittance_params.remitter
    self.remitter_contact_info = remittance_params.remitter_contact_info
    return True

# 行動支付實作
class MobilePayment(PaymentMethod):
  def __init__(self):
    super().__init__()
    self.payment_name = "MobilePayment"
    self.mobile_payment_name = ""

  def pay(self, parameters: IPaymentParameters):
    mobilepay_parameters = parameters
    # 保存行動支付方式
    self.mobile_payment_name = mobilepay_parameters.mobilepay.mobile_payment_name
    print(f"\n行動支付成功，支付方式：{self.mobile_payment_name}")
    return mobilepay_parameters.mobilepay.pay()

In [85]:
# 訂單類別
class Order:
  def __init__(self, order_id, order_payment_method, order_amount):
    print("==================================")
    print("新增訂單...")

    #設定訂單內容
    self.order_id = int(time.strftime("%Y%m%d", time.localtime())) + order_id
    self.order_payment_method = order_payment_method
    self.payment_status = "Pending"
    self.create_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    self.order_amount = order_amount

    #列印訂單內容
    print(f"訂單編號：{self.order_id}")
    print(f"訂單日期：{self.create_time}")
    print(f"支付方式：{self.order_payment_method.payment_name}")

    #匯款支付顯示收款帳號
    if isinstance(self.order_payment_method, RemittancePayment):
      print("\n請將款項匯至以下帳戶")
      print(f"銀行代碼：{self.order_payment_method.receiving_bank}")
      print(f"收款帳號：{self.order_payment_method.receiving_account_number}")

  def set_payment_method(self, payment_method):
    self.order_payment_method = payment_method

  def process_payment(self, parameters):
    if self.order_payment_method.pay(parameters):
      self.update_payment_status("Completed")
    else:
      self.update_payment_status("Failed")

  def update_payment_status(self, status):
    self.payment_status = status
    print(f"\n更新訂單付款狀態為：{status}")
    return True

  def set_recipient_info(self, recipient_name, recipient_address, recipient_phone):
    self.recipient_name = recipient_name
    self.recipient_address = recipient_address
    self.recipient_phone = recipient_phone

    # 列印收件人資訊
    print("\n設定收件人資訊")
    print(f"收件人：{self.recipient_name}")
    print(f"收件地址：{self.recipient_address}")
    print(f"收件電話：{self.recipient_phone}")

In [86]:
# 付款測試
orders = []

# 貨到付款
order = Order(len(orders), CashOnDelivery(),2000)
order.set_recipient_info("陳大明","高雄市苓雅區五福路100號","0912456999")
orders.append(order)
payment_parameters = CashOnDeliveryParameters()
order.process_payment(payment_parameters)

# 信用卡支付
order = Order(len(orders), CreditCardPayment(),1200)
order.set_recipient_info("黃大白","高雄市苓雅區和平路100號","0972458959")
orders.append(order)
payment_parameters = CreditCardPaymentParameters("1234-5678-9012-9870", 26, 3, "123")
order.process_payment(payment_parameters)

# 匯款支付
order = Order(len(orders), RemittancePayment(),1000)
order.set_recipient_info("張小花","高雄市左營區博愛二路100號","0912345678")
orders.append(order)
payment_parameters = RemittancePaymentParameters("78901", "張小花", "0912345678")
order.process_payment(payment_parameters)

# 行動支付-LinePay
order = Order(len(orders), MobilePayment(),2300)
order.set_recipient_info("顏阿光","高雄市左營區崇德路101號","0977745655")
orders.append(order)
payment_parameters = MobilePaymentParameters(LinePay())
order.process_payment(payment_parameters)

# 行動支付-JkoPay
order = Order(len(orders), MobilePayment(),1500)
order.set_recipient_info("謝黃高","高雄市左營區裕誠路99號","0978889123")
orders.append(order)
payment_parameters = MobilePaymentParameters(JkoPay())
order.process_payment(payment_parameters)


print("==================================")

新增訂單...
訂單編號：20241212
訂單日期：2024-12-12 07:56:36
支付方式：CashOnDelivery

設定收件人資訊
收件人：陳大明
收件地址：高雄市苓雅區五福路100號
收件電話：0912456999

收到款項

更新訂單付款狀態為：Completed
新增訂單...
訂單編號：20241213
訂單日期：2024-12-12 07:56:36
支付方式：CreditCardPayment

設定收件人資訊
收件人：黃大白
收件地址：高雄市苓雅區和平路100號
收件電話：0972458959

付款成功信用卡後四碼：9870

更新訂單付款狀態為：Completed
新增訂單...
訂單編號：20241214
訂單日期：2024-12-12 07:56:36
支付方式：RemittancePayment

請將款項匯至以下帳戶
銀行代碼：800
收款帳號：1234567890

設定收件人資訊
收件人：張小花
收件地址：高雄市左營區博愛二路100號
收件電話：0912345678

回報匯款資訊
匯款帳號後五碼：78901
匯款人姓名：張小花
匯款人聯絡方式：0912345678

更新訂單付款狀態為：Completed
新增訂單...
訂單編號：20241215
訂單日期：2024-12-12 07:56:36
支付方式：MobilePayment

設定收件人資訊
收件人：顏阿光
收件地址：高雄市左營區崇德路101號
收件電話：0977745655

行動支付成功，支付方式：LinePay

更新訂單付款狀態為：Completed
新增訂單...
訂單編號：20241216
訂單日期：2024-12-12 07:56:36
支付方式：MobilePayment

設定收件人資訊
收件人：謝黃高
收件地址：高雄市左營區裕誠路99號
收件電話：0978889123

行動支付成功，支付方式：JkoPay

更新訂單付款狀態為：Completed
